In [1]:
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import string

In [2]:
df_original=pd.read_csv("../data/initial_merged_dataset.csv")

In [3]:
df_original

,name,link,image,category,store,2023-07-30,2023-08-15,2023-08-24
0,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,10580,10580,10580
1,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,10580,10580,10580
2,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,10580,10580,10580
3,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,11980,11980,11980
4,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,11980,11980,11980
...,...,...,...,...,...,...,...,...
3792,GORENJE FH-301CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288320/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,22999,22999,22999
3793,GORENJE FH-401CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288321/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,23999,23999,23999
3794,INDESIT UI-6 1 W ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/740637/inde...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,27999,27999,27999
3795,WHIRLPOOL WHE-3133.1 ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29289082/wh...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,28999,28999,28999


In [4]:
df=df_original.copy()

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\\', ' ').replace('/', ' ')
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

In [6]:
df['name_preprocessed'] = df['name'].apply(preprocess_text)

In [7]:
df

,name,link,image,category,store,2023-07-30,2023-08-15,2023-08-24,name_preprocessed
0,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,10580,10580,10580,notebook acer aspire a11522 3020e 4gb 128gb 15...
1,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,10580,10580,10580,notebook acer aspire a11522 3020e 4gb 128gb 15...
2,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,10580,10580,10580,notebook acer aspire a11522 3020e 4gb 128gb 15...
3,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,11980,11980,11980,notebook lenovo ip3 3020e 4gb 256gb ssd vega3 ...
4,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,Laptop,Anhoch,11980,11980,11980,notebook lenovo ip3 3020e 4gb 256gb ssd vega3 ...
...,...,...,...,...,...,...,...,...,...
3792,GORENJE FH-301CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288320/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,22999,22999,22999,gorenje fh301cw замрзнувач
3793,GORENJE FH-401CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288321/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,23999,23999,23999,gorenje fh401cw замрзнувач
3794,INDESIT UI-6 1 W ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/740637/inde...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,27999,27999,27999,indesit ui6 1 w замрзнувач
3795,WHIRLPOOL WHE-3133.1 ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29289082/wh...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,Freezers,TehnoMarket,28999,28999,28999,whirlpool whe31331 замрзнувач


In [8]:
nlp = spacy.load("..\\venv\Lib\site-packages\en_core_web_md\en_core_web_md-3.6.0")

In [9]:
def get_word_embeddings(row):
    doc = nlp(row['name_preprocessed'])
    return doc.vector

In [10]:
df['vectorized_name'] = df.apply(get_word_embeddings, axis=1)

In [11]:
similarity_matrix = cosine_similarity(np.array(df['vectorized_name'].values.tolist()))

In [12]:
similarity_matrix

array([[ 1.0000002 ,  1.0000002 ,  1.0000002 , ...,  0.36691955,
        -0.0396841 , -0.0396841 ],
       [ 1.0000002 ,  1.0000002 ,  1.0000002 , ...,  0.36691955,
        -0.0396841 , -0.0396841 ],
       [ 1.0000002 ,  1.0000002 ,  1.0000002 , ...,  0.36691955,
        -0.0396841 , -0.0396841 ],
       ...,
       [ 0.36691955,  0.36691955,  0.36691955, ...,  0.9999997 ,
         0.03756174,  0.03756174],
       [-0.0396841 , -0.0396841 , -0.0396841 , ...,  0.03756174,
         1.        ,  1.        ],
       [-0.0396841 , -0.0396841 , -0.0396841 , ...,  0.03756174,
         1.        ,  1.        ]], dtype=float32)

In [13]:
agg_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.01)
cluster_labels = agg_clustering.fit_predict(similarity_matrix)

In [14]:
df['cluster_id']=cluster_labels

In [15]:
df['vectorized_name_json'] = df['vectorized_name'].apply(lambda x: pd.Series(x).to_json(orient='values'))

In [18]:
def map_categories(row):
    cat=row['category']
    if cat=='Laptop':
        return "LAPTOP"
    elif cat=="Phone":
        return "PHONE"
    elif cat=="Air Conditioner":
        return "AC"
    elif cat=="Fridge":
        return "FRIDGE"
    elif cat=="Freezers":
        return "FREEZER"
    else:
        return cat
    # LAPTOP, PHONE, TV, CPU, GPU, AC, FRIDGE, FREEZER

In [19]:
df['category']=df.apply(map_categories,axis=1)

In [27]:
columns=df.columns
first_columns = columns[0:5]
last_columns = columns[-4:]
middle_columns = columns[5:8]
df=df[first_columns].join(df[last_columns]).join(df[middle_columns])

In [28]:
df

,name,link,image,category,store,name_preprocessed,vectorized_name,cluster_id,vectorized_name_json,2023-07-30,2023-08-15,2023-08-24
0,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,LAPTOP,Anhoch,notebook acer aspire a11522 3020e 4gb 128gb 15...,"[-0.06938505, 0.17586645, -2.1457279, 1.868082...",460,"[-0.0693850517,0.1758664548,-2.1457278728,1.86...",10580,10580,10580
1,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,LAPTOP,Anhoch,notebook acer aspire a11522 3020e 4gb 128gb 15...,"[-0.06938505, 0.17586645, -2.1457279, 1.868082...",460,"[-0.0693850517,0.1758664548,-2.1457278728,1.86...",10580,10580,10580
2,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,LAPTOP,Anhoch,notebook acer aspire a11522 3020e 4gb 128gb 15...,"[-0.06938505, 0.17586645, -2.1457279, 1.868082...",460,"[-0.0693850517,0.1758664548,-2.1457278728,1.86...",10580,10580,10580
3,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,LAPTOP,Anhoch,notebook lenovo ip3 3020e 4gb 256gb ssd vega3 ...,"[0.3066337, 0.7494219, -2.3564832, 1.2007939, ...",848,"[0.3066337109,0.7494218946,-2.3564832211,1.200...",11980,11980,11980
4,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,LAPTOP,Anhoch,notebook lenovo ip3 3020e 4gb 256gb ssd vega3 ...,"[0.3066337, 0.7494219, -2.3564832, 1.2007939, ...",848,"[0.3066337109,0.7494218946,-2.3564832211,1.200...",11980,11980,11980
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,GORENJE FH-301CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288320/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,FREEZER,TehnoMarket,gorenje fh301cw замрзнувач,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",367,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",22999,22999,22999
3793,GORENJE FH-401CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288321/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,FREEZER,TehnoMarket,gorenje fh401cw замрзнувач,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",367,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",23999,23999,23999
3794,INDESIT UI-6 1 W ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/740637/inde...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,FREEZER,TehnoMarket,indesit ui6 1 w замрзнувач,"[-0.67712, -0.9719, 0.5978558, 1.8894199, 1.35...",773,"[-0.6771199703,-0.9718999863,0.5978558064,1.88...",27999,27999,27999
3795,WHIRLPOOL WHE-3133.1 ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29289082/wh...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,FREEZER,TehnoMarket,whirlpool whe31331 замрзнувач,"[-0.09362999, 0.31618333, -0.21285333, -0.674,...",129,"[-0.0936299935,0.3161833286,-0.2128533274,-0.6...",28999,28999,28999


In [29]:
df.to_csv("initial_data_preprocessed.csv", index=False)